In [1]:
import sys
sys.path.append('..')

import torch
import numpy as np
from torch.utils.data import DataLoader

from evaluate.load.helpers import *
from evaluate.load.dataset import *
from evaluate.pipeline.model import *
from evaluate.pipeline.trainer import *
from evaluate.pipeline.embedding import *

In [2]:
def get_param(mode="paper",isHashEmbedding=True):
    return {"use_hash_embeddings": isHashEmbedding,
            "hashing_trick": True,
            "ngram_range": (1,9),
            "nFeaturesRange": (4,100),
            "embedding_size": 20,
            "num_buckets": 10**6,
            "max_features": 10**7,
            "max_epochs": 100,
            "num_hash_functions": 2,
            "hidden": 50,
            "seed": 123,
            "batch_size": 32,
            "masking":True,
            "append_weight":False,
            "validation_size":0.05,
            "patience":10,
            'cuda':True}
            
param = get_param()
seed = param['seed']
max_features = param['max_features']
ngram_range = param['ngram_range']
use_hash_embeddings = param['use_hash_embeddings']
isHashingTrick = param['hashing_trick']
num_buckets = param['num_buckets']
embedding_size = param['embedding_size']
max_epochs = param['max_epochs']
masking = param['masking']
append_weight = param['append_weight']
patience = param['patience']
num_hash_functions = param['num_hash_functions']
batch_size = param['batch_size']
validation_size = param['validation_size']
nFeaturesRange = param['nFeaturesRange']
isCuda = param['cuda']

In [ ]:
%%time

np.random.seed(seed)
torch.manual_seed(seed)

train = AgNews(nFeaturesRange=nFeaturesRange,maxFeatures=max_features,train=True,ngram_range=ngram_range,isHashingTrick=isHashingTrick,seed=seed)
test = AgNews(nFeaturesRange=nFeaturesRange,maxFeatures=max_features,train=False,ngram_range=ngram_range,isHashingTrick=isHashingTrick,seed=seed)

In [ ]:
#train, valid = train_valid_load(train,validSize=0.1,isShuffle=True,seed=123,batch_size=batchSize)
test = DataLoader(dataset=test,batch_size=batch_size,shuffle=True)

In [ ]:
num_classes = len(train.classes)
model = ModelNoDict(max_features,embedding_size,num_classes,isHash=True,append_weight=True,num_buckets=num_buckets)
if isCuda:
    model.cuda
trainer = Trainer(model,seed=seed)
callbacks = [EarlyStopping(patience=patience)]

In [ ]:
%%time
trainer(train,callbacks=callbacks,validSize=validation_size)

In [6]:
%%time
trainer(train,callbacks=callbacks,validSize=validation_size)

Num parameters in model: 40000092
Train on 3563 samples, validate on 188 samples
Epoch: 0. Loss: 0.533701479434967. Acc: 0.7716666666666666.
Epoch: 1. Loss: 0.24999885261058807. Acc: 0.782.
Epoch: 2. Loss: 0.13356901705265045. Acc: 0.794.
Epoch: 3. Loss: 0.20342786610126495. Acc: 0.7993333333333333.
Epoch: 4. Loss: 0.34462618827819824. Acc: 0.7976666666666666.
Epoch: 5. Loss: 0.09012521803379059. Acc: 0.7925.
Epoch: 6. Loss: 0.14708219468593597. Acc: 0.7915.


KeyboardInterrupt: 

In [7]:
num_classes = len(train.classes)
model = ModelNoDict(max_features,embedding_size,num_classes,isHash=False,num_buckets=num_buckets)
trainer = Trainer(model)
callbacks = [EarlyStopping(patience=param['patience'])]

In [8]:
%%time
trainer(train,callbacks=callbacks,validSize=validation_size)

Num parameters in model: 200000084
Train on 3563 samples, validate on 188 samples


KeyboardInterrupt: 

Exception ignored in: 'zmq.backend.cython.message.Frame.__dealloc__'
Traceback (most recent call last):
  File "zmq/backend/cython/checkrc.pxd", line 12, in zmq.backend.cython.checkrc._check_rc
KeyboardInterrupt: 


KeyboardInterrupt: 

In [55]:
%%time
trainer(train,callbacks=callbacks,validSize=0.1)

Num parameters in model: 2000084
Train on 3375 samples, validate on 375 samples
Epoch: 0. Loss: 0.16543151438236237. Acc: 0.9149166666666667.
Epoch: 1. Loss: 0.2636685371398926. Acc: 0.9113333333333333.
Epoch: 2. Loss: 0.11431225389242172. Acc: 0.90375.
Stoping at epoch:2 with patience:2. Best:0.9149166666666667.
CPU times: user 5min 55s, sys: 1.13 s, total: 5min 56s
Wall time: 5min 57s
